In [1]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import math
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/analysis_lib.ipynb"

In [2]:
args = getargs()

/home/ibis/gregor.sturm/bin/anaconda/lib/python2.7/site-packages/nbwrapper.py:31: RuntimeWarning: no arguments passed!
  warnings.warn("no arguments passed!", RuntimeWarning)


In [3]:
NMERS = 6

In [4]:
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.called",
    "ref": "/home/ibis/gregor.sturm/nanopore/NanoporeData/PublicData/LomanLab_MAP-006/ecoli_mg1655.fa",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1.model.pickle",
    "corr_model": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/context_prediction/model-test2.pickle",
    "ncores": 62,
    "nmers": NMERS,
    "multivariate": False
}

In [5]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [6]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = HMM_PARAMS["/opt/chimaera/model/r7.3_e6_70bps_6mer/template_median68pA.model"]
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS["kmer"].tolist() == ALL_KMERS

# Train Model 

In [ ]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [ ]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [ ]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (2/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (47/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [ ]:
mk_transmat = mk_transmat2

In [ ]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = HMM_PARAMS[["level_mean", "level_stdv"]].values.tolist() #mu, std of each state
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state
    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [ ]:
F = ghmm.Float()  # emission domain of this model
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [ ]:
model = mk_model()
s = str(model)
print(s)

In [ ]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[0] for kmer in kmers] + [kmers[-1][1:]]
    return "".join(seq)

In [ ]:
def predict(events):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    emissions = [x[0] for x in events]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [ ]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [ ]:
predict(seq)

In [ ]:
model

## multistep-prediction

In [ ]:
from sklearn import ensemble
import joblib
import mltools

In [ ]:
corr_model = joblib.load(args["corr_model"])

In [ ]:
OFFSET = 2

# Validate Model 

In [ ]:
!pwd

In [24]:
assert os.path.isfile(args["events"])

In [25]:
ref = load_ref(args["ref"])

['>gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome']
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT


In [59]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [60]:
prepare_filemap(file_data)

In [28]:
def basecall_read(file_obj):
    file_obj = correct_read(file_obj, col="mean")
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict_iterative(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [29]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [61]:
file_obj = file_data[3]

In [62]:
file_obj = correct_read(file_obj, col="mean")
events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]

In [63]:
seq = predict(events)

In [39]:
means = [x[0] for x in events]

In [87]:
corr_range = (OFFSET, len(seq)-OFFSET-NMERS)
print (">> started feature generation")
features = []
for i in range(*corr_range): 
    print(i, seq[i-OFFSET:i], seq[i+NMERS:i+NMERS+OFFSET])
    print(mltools.seq2binvec(seq[i-OFFSET:i] + seq[i+NMERS:i+NMERS+OFFSET]))
    features.append(mltools.seq2binvec(seq[i-OFFSET:i] + seq[i+NMERS:i+NMERS+OFFSET]))
    if i > 12:
          break

>> started feature generation
(2, 'TG', 'TT')
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
(3, 'GT', 'TG')
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
(4, 'TC', 'GG')
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
(5, 'CG', 'GT')
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
(6, 'GT', 'TG')
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
(7, 'TG', 'GC')
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
(8, 'GT', 'CT')
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
(9, 'TT', 'TG')
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
(10, 'TT', 'GA')
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
(11, 'TG', 'AT')
[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]
(12, 'GG', 'TA')
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]
(13, 'GT', 'AT')
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [89]:
seq[:12]

'TGTCGTGTTTGG'

In [91]:
correction = corr_model.predict(features)  

In [94]:
correction

array([ 0.22880741,  0.1503749 , -0.30079442, -0.09556875,  0.1503749 ,
        0.04893353, -0.14575582,  0.06788583, -0.45160604,  0.13059853,
        0.10723831,  0.15565685])

In [96]:
means[:12]

[39.875358168629404,
 37.590656708989044,
 31.005801483921676,
 30.971203794443309,
 28.342124992244202,
 32.720483781372479,
 39.199280465681454,
 36.583741536485654,
 36.116147229960035,
 33.1147591176234,
 36.416133531635154,
 44.160416405433018]

In [70]:
for i, j  in enumerate(range(*corr_range)):
    means[j] -= .5*correction[i]

In [71]:
means[202:212]

[37.573281123680736,
 44.047762646724188,
 51.793498320381538,
 47.704320915219931,
 49.122656047534157,
 41.099816484309208,
 28.138853414434969,
 27.277070004509241,
 26.25595199973856,
 31.736987765649367]

'TGTCGTGTTTGG'